In [19]:
import pandas as pd
import os
from tqdm import tqdm
import json
from utils import *
# replace it with your downloaded folder path
full_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-full-text'
meta_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-meta'
result_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-results'

In [53]:
import openai
from openai import OpenAI
import yaml
with open("/Users/junyi/Work/RR/config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)
openai.api_key = config["openai_api_key"]
client = OpenAI(api_key=config["openai_api_key"])

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

import string

def correct_url(url):
    return url.rstrip(string.punctuation)
corrected_url = correct_url(url)
corrected_url

def fetch_redirected_url_with_selenium(doi_url):
    try:
        # Set up Selenium WebDriver with headless mode
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')

        service = Service()  # Ensure ChromeDriver is correctly installed or provide the path
        driver = webdriver.Chrome(service=service, options=options)

        driver.get(doi_url)
        time.sleep(5)  # Allow time for redirection if needed

        redirected_url = driver.current_url
        driver.quit()
        return redirected_url
    except Exception as e:
        return f"Error: {str(e)}"
    
def is_dataset_link(url, client=client):
    """
    Determines if a given URL is a dataset link.

    Args:
        client (OpenAI): The OpenAI client instance.
        url (str): The URL to evaluate.

    Returns:
        bool: True if the URL is a dataset link, False otherwise.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Is this a link to host the dataset by university?" + url
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "my_schema",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "is_dataset_link": {
                            "type": "boolean"
                        }
                    },
                    "required": [
                        "is_dataset_link"
                    ],
                    "additionalProperties": False
                }
            }
        },
        temperature=0,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # Extract and return the result
    return response.choices[0].message.content


In [21]:
data = pd.read_csv(os.path.join(meta_data_folder, 'full-meta-dataset.csv'))
data.columns

Index(['title', 'doi', 'volume', 'date', 'year', 'month', 'abstract', 'issn',
       'journal_name', 'unique_id', 'is_github', 'num_of_github_urls',
       'is_availablity_statement', 'is_data_mentioned_in_section_title',
       'is_experiment_mentioned_in_section_title',
       'is_link_in_avaiablity_statement',
       'num_of_links_in_avaiablity_statement'],
      dtype='object')

In [22]:
if not os.path.exists(result_data_folder + '/data-links'):
    os.makedirs(result_data_folder + '/data-links')
# Regular expression to find URLs
url_pattern = r'https?://[^\s\)\.]+(?:[^\s\)]+)' # Matches 'http://' or 'https://' followed by non-whitespace characters
full_url_list = []
for i in tqdm(range(len(data))):
# for i in tqdm(range(2000, 2101)):
    # Initialize data context
    data_context = ''
    label = 0
    try:
        # Construct paths for journal and paper
        journal_path = os.path.join(full_data_folder, data['issn'][i])
        paper_path = os.path.join(journal_path, f"{data['unique_id'][i]}.xml")
        
        # get the abstract from the xml
        abstract = extract_abstract_from_xml(paper_path)
        if abstract:
            # if "data" in abstract.lower():
            if "data" in abstract.lower():
                label = 1
        data_context += (abstract + '\n')
        # Extract and process sections from the paper XML
        sections = extract_sections_and_text_from_xml(paper_path)
        reorganized_sections = postprocess_sections(sections)

        # Traverse the sections and collect data-related text
        for section in reorganized_sections:
            if 'data' in section['title'].lower() and 'supplementary' not in section['title'].lower():
                data_context += (section['text'] + '\n')
                label = 1
            for subsection in section.get('subsections', []):
                if 'data' in subsection['title'].lower():
                    data_context += (subsection['text'] + '\n')
                    label = 1
                for subsubsection in subsection.get('subsubsections', []):
                    if 'data' in subsubsection['title'].lower():
                        data_context += (subsubsection['text'] + '\n')
                        label = 1
    except KeyError as e:
        print(f"Missing key in data: {e}")
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    if data_context:
        # Find all URLs in the text
        urls = re.findall(url_pattern, data_context)
        if urls:
            print(f"Found {len(urls)} URLs in the text")
            full_url_list.extend(urls)
            # Save the URLs to a file
            with open(os.path.join(result_data_folder, 'data-links', f"{data['unique_id'][i]}.txt"), 'w') as f:
                for url in urls:
                    f.write(url + '\n')
            # check if doi.org is in the url
            for url in urls:
                # result = is_dataset_link(url)
                if (
                    'doi.org' in url
                    or 'osf.io' in url
                    or 'figshare.com' in url
                    or 'data.mendeley.com' in url
                    or 'zenodo' in url
                    or 'dataverse' in url
                    or 'github' in url
                    or 'dataverse.harvard.edu' in url
                    or 'data-in-brief' in url
                    or 'datadryad.org' in url
                ):
                    print(f"Found DOI URL: {url}")

  0%|          | 0/10990 [00:00<?, ?it/s]

  0%|          | 41/10990 [00:00<01:58, 92.20it/s]

Found 1 URLs in the text
Found 1 URLs in the text


  1%|          | 126/10990 [00:01<01:48, 100.03it/s]

Found 1 URLs in the text
Found 2 URLs in the text
Found DOI URL: https://github.com/numenta/NAB/tree/master/data


...

100%|█████████▉| 10972/10990 [05:10<00:00, 50.59it/s]

Found 2 URLs in the text
Found 1 URLs in the text
Found DOI URL: https://github.com/ibatur/atus_heat_paper.
Found 1 URLs in the text


100%|█████████▉| 10988/10990 [05:10<00:00, 45.69it/s]

Found 1 URLs in the text
Found DOI URL: https://doi.org/10.4121/777220e4-9a78-4653-8780-89c2572f6c81.
Found 1 URLs in the text
Found 2 URLs in the text
Found 1 URLs in the text


100%|██████████| 10990/10990 [05:10<00:00, 35.36it/s]


In [19]:
with open(os.path.join(result_data_folder, 'data-links', f"{data['unique_id'][i]}.txt"), 'w') as file:
	file.write(data_context)

In [ ]:
# save the full url list
with open(os.path.join(result_data_folder, 'data-links', 'full-url-list.txt'), 'w') as f:
    for url in full_url_list:
        f.write(url + '\n')

In [64]:
# transform str to json
import json
result = json.loads(result)

result['is_dataset_link']

True

In [17]:
import re

# Pattern to match URLs
url_pattern = r'(https?://[A-Za-z0-9.\-_/]+)(?=$|\s|[.,\)])'
text = "Found DOI URL: https://github.com/RRinTransportation/rr-measure-basic/. The Check https://example.com/path) for more information."

# Find URLs
urls = re.findall(url_pattern, text)

# Clean up trailing parentheses and periods
clean_urls = [re.sub(r'[)\.]+$', '', url) for url in urls]
clean_urls

['https://github.com/RRinTransportation/rr-measure-basic/',
 'https://example.com/path']

In [54]:
import re
from collections import Counter
from urllib.parse import urlparse
import requests

# Load the URL list from the file
file_path = os.path.join(result_data_folder, 'data-links', 'full-url-list.txt')
with open(file_path, 'r') as file:
    urls = file.readlines()

# Remove whitespace characters
urls = [url.strip() for url in urls]

# Extract domain, protocol, and resource type
domains = []
protocols = []
resource_types = []

for url in urls:
    url = correct_url(url)
    if 'doi.org' in url:
        url = fetch_redirected_url_with_selenium(url)
        print(f"Redirected URL: {url}")
    parsed_url = urlparse(url)
    domains.append(parsed_url.netloc)
    protocols.append(parsed_url.scheme)
    
    # Identify resource type (e.g., API, PDF, general website)
    if parsed_url.path.endswith('.pdf'):
        resource_types.append('PDF')
    elif 'api' in parsed_url.path or 'api' in parsed_url.netloc:
        resource_types.append('API')
    else:
        resource_types.append('Website')

# Count occurrences
domain_counts = Counter(domains)
protocol_counts = Counter(protocols)
resource_type_counts = Counter(resource_types)

# Print insights
print("Domain Distribution:")
for domain, count in domain_counts.most_common(10):
    print(f"{domain}: {count}")

print("\nProtocol Analysis:")
for protocol, count in protocol_counts.items():
    print(f"{protocol}: {count}")

print("\nResource Type Analysis:")
for resource_type, count in resource_type_counts.items():
    print(f"{resource_type}: {count}")


Redirected URL: https://osf.io/de8km/
Redirected URL: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VGMDPC
Redirected URL: https://data.mendeley.com/datasets/tfkgpmgf9t/1
Redirected URL: https://figshare.com/articles/software/Data_and_code_for_FDA_analysis/13171862
Redirected URL: https://data.mendeley.com/datasets/5gy6s8trbm/2
Redirected URL: https://figshare.com/articles/dataset/Pedestrian_moral_disengagement_scale_for_sustainable_urban_planning/20311812/2
Redirected URL: https://figshare.com/articles/software/Snapshot_of_GitHub_Repository_to_accompany_paper_b_Evaluation_of_bicycle_sharing_scheme_data_as_a_proxy_for_cycling_mobility_-_how_COVID-19_measures_influenced_cycling_in_Paris_b_/24231706?file=42570667
Redirected URL: https://datadryad.org/stash/dataset/doi:10.25338/B8035D
Redirected URL: https://data.4tu.nl/datasets/e4324d0c-2a82-4f03-8cbb-64dd207cd522
Redirected URL: https://accounts.osf.io/login?service=https%3A%2F%2Fosf.io%2Fx7aer%2F
Redirected U

In [56]:
# Print insights
print("Domain Distribution:")
for domain, count in domain_counts.items():
    print(f"{domain}: {count}")

print("\nProtocol Analysis:")
for protocol, count in protocol_counts.items():
    print(f"{protocol}: {count}")

print("\nResource Type Analysis:")
for resource_type, count in resource_type_counts.items():
    print(f"{resource_type}: {count}")

Domain Distribution:
bitre.gov.au: 1
www.nco.ncep.noaa.gov: 1
cdn.wartsila.com: 1
github.com: 111
cran.r-project.org: 2
www.afrra.org: 1
www.who.int: 4
www.worldometers.info: 1
www.google.com: 9
covid19.ncdc.gov.ng: 1
osf.io: 29
dataverse.harvard.edu: 4
netlipse.eu: 1
www.wko.at: 1
www.aviationreg.ie: 2
www.youtube.com: 5
factfinder.census.gov: 1
www.census.gov: 11
pedbikeinfo.org: 3
www.covidmobilityworks.org: 1
datasketch.github.io: 1
www.sfcta.org: 1
cycleatlanta.org: 1
www.thempo.org: 1
maps.nyc.gov: 1
covid19-dashboard.ages.at: 1
www.norgeshelsa.no: 1
www.innocom.gov.cn: 1
www.qilib.com: 1
www.udparty.com: 1
www.stats.gov.cn: 4
covid19.apple.com: 1
covid19.saglik.gov.tr: 1
drive.google.com: 3
map.baidu.com: 4
dianshi.baidu.com: 1
data.worldbank.org: 3
asrs.arc.nasa.gov: 1
data.covid.umd.edu: 3
openaq.org: 1
www.icao.int: 4
jeodpp.jrc.ec.europa.eu: 1
www.mrlc.gov: 1
www.kaggle.com: 3
www.statistics.gr: 1
data.mendeley.com: 13
figshare.com: 4
www.ncei.noaa.gov: 3
c2smart.engineering

In [31]:
response.url

'https://data.4tu.nl/articles/_/12681650/1'

In [55]:
print("Domain Distribution:")
for domain, count in domain_counts.most_common(20):
    print(f"{domain}: {count}")

Domain Distribution:
github.com: 111
osf.io: 29
www.openstreetmap.org: 20
zenodo.org: 17
data.4tu.nl: 16
lbs.amap.com: 16
www.wjx.cn: 14
data.cityofchicago.org: 14
www1.nyc.gov: 14
data.mendeley.com: 13
www.census.gov: 11
www.google.com: 9
pems.dot.ca.gov: 8
www.12306.cn: 7
www.ssb.no: 7
www.citibikenyc.com: 7
gaia.didichuxing.com: 7
www.wunderground.com: 6
www.transtats.bts.gov: 6
developers.google.com: 6


In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

def fetch_redirected_url_with_selenium(doi_url):
    try:
        # Set up Selenium WebDriver with headless mode
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')

        service = Service()  # Ensure ChromeDriver is correctly installed or provide the path
        driver = webdriver.Chrome(service=service, options=options)

        driver.get(doi_url)
        time.sleep(5)  # Allow time for redirection if needed

        redirected_url = driver.current_url
        driver.quit()
        return redirected_url
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
doi_url = "https://doi.org/10.17605/OSF.IO/DE8KM."

doi_url = correct_url(doi_url)
redirected_url = fetch_redirected_url_with_selenium(doi_url)
print("Redirected URL:", redirected_url)

Redirected URL: https://osf.io/de8km/


In [49]:
import string

def correct_url(url):
    return url.rstrip(string.punctuation)
corrected_url = correct_url(url)
corrected_url

'https://doi.org/10.17632/tfkgpmgf9t.1'